In [72]:
import pandas as pd
import numpy as np

import yfinance as yf

import matplotlib.pyplot as plt
from datetime import datetime

import math

from pathlib import Path  
filepath = 'D:/Misha/Documents/MSc Thesis/' 

import itertools

from pandas.tseries.offsets import BDay

import nasdaqdatalink

from scipy.optimize import fsolve
from scipy.stats import norm

import wrds
db = wrds.Connection(wrds_username='mborovkov')

Loading library list...
Done


In [73]:
sp500_hist = pd.read_csv('SP500Hist.csv', sep = ',')
sp500_hist.tickers = sp500_hist.tickers.str.replace('"','')

sp500_hist.tickers = sp500_hist.tickers.str.split(',')

import itertools
sp500_ticker_list = tuple(set(itertools.chain.from_iterable(sp500_hist.tickers)))

sp500_secid_df = db.raw_sql(f'''select secid, ticker
                    FROM optionm_all.secnmd
                    WHERE ticker IN {sp500_ticker_list}
                    ''').drop_duplicates().reset_index(drop = True)

sp500_secid_df = sp500_secid_df[sp500_secid_df['secid'] > 99999]

In [79]:
#import nasdaqdatalink
#zcb = nasdaqdatalink.get("FED/SVENY", start_date="1997-01-01", end_date="2021-12-31")['SVENY01']
zcb = pd.read_csv('zcb.csv')
zcb['Date'] = pd.to_datetime(zcb['Date'], format='%m/%d/%Y')   
    
zcb

,Date,SVENY01
0,1997-01-02,5.6353
1,1997-01-03,5.6216
2,1997-01-06,5.6221
3,1997-01-07,5.6280
4,1997-01-08,5.6457
...,...,...
6243,2021-12-27,0.4172
6244,2021-12-28,0.4207
6245,2021-12-29,0.4146
6246,2021-12-30,0.4186


In [80]:
import warnings
warnings.filterwarnings('ignore')

def calc_upper_skew(df, date, fwd_price, bond):
    df = df.sort_values('strike_price')
    df['diff'] = df['strike_price'].diff()
    df = df.reset_index(drop = True)
    if len(df) > 0:
        df['diff'].iloc[0] = df['strike_price'].iloc[0] - fwd_price
        df['upper_skew_comp'] = np.log(df['strike_price']/fwd_price) * np.sqrt(df['strike_price']/fwd_price) * df['price'] * df['diff'] / (df['strike_price'] * df['strike_price'])
        row = df['upper_skew_comp'].groupby(df['date']).sum().reset_index()
        return row['upper_skew_comp'][0] * 6 * (1+bond/100)**0.25 # CHANGE FOR DESIRED MATURITY
    return 1000

def calc_lower_skew(df, date, fwd_price, bond):
    df = df.sort_values('strike_price', ascending = False)
    df['diff'] = df['strike_price'].diff()
    df = df.reset_index(drop = True)
    if len(df) > 0:
        df['diff'].iloc[0] = df['strike_price'].iloc[0] - fwd_price
        df = df.sort_values('strike_price')
        df['lower_skew_comp'] = np.log(fwd_price/df['strike_price']) * np.sqrt(df['strike_price']/fwd_price) * df['price'] * abs(df['diff']) / (df['strike_price'] * df['strike_price'])
        row = df['lower_skew_comp'].groupby(df['date']).sum().reset_index()
        return row['lower_skew_comp'][0] * -6 * (1+bond/100)**0.25 # CHANGE FOR DESIRED MATURITY
    return 1000
    
def calc_upper_and_lower_skew(df):
    date = df['date'].iloc[0]
    secid = df['secid'].iloc[0]
    fwd_price = df['fwd_price'].iloc[0]
    bond = df['SVENY01'].iloc[0]    
    df = df[df['days'] == df['days'].min()]
    
    df_c = df.loc[df['cp_flag'] == 'C']
    df_p = df.loc[df['cp_flag'] == 'P']
    usk = calc_upper_skew(df_c, date, fwd_price, bond)
    lsk = calc_lower_skew(df_p, date, fwd_price, bond)
    return [date, secid, usk, lsk, df['days'].min()]

#calc_upper_and_lower_skew(oppr_stock[0:52])

In [81]:
%%time
years = list(range(1997, 2022, 1))
#years = [2020, 2021]
#secid_list = [101594]
secid_list = tuple(sp500_secid_df.secid)
skew_df = pd.DataFrame(columns=['date', 'secid', 'upper_skew', 'lower_skew', 'days'])

for secid in secid_list:
    oppr_stock =  pd.DataFrame(columns=['secid', 'date', 'exdate', 'cp_flag', 'delta', 'strike_price', 'best_bid', 'best_offer'])
    fwd =  pd.DataFrame(columns=['date', 'days', 'delta', 'impl_volatility', 'secid', 'impl_strike'])
    for year in years:
        temp = db.raw_sql(f'''select secid, date, exdate, cp_flag, delta, strike_price, best_bid, best_offer
                        FROM optionm_all.opprcd{year} 
                        WHERE secid = {secid}
                        ''')
        
        oppr_stock = pd.concat([oppr_stock, temp])
        
        # CHANGE 91 TO DESIRED MATURITY
        temp = db.raw_sql(f'''select date, days, delta, impl_volatility, secid, impl_strike 
                        FROM optionm_all.vsurfd{year} 
                        WHERE secid = {secid}
                        AND delta in (-50, 50)
                        AND days = 91 
                        ''')
        fwd = pd.concat([fwd, temp])
        
    fwd = fwd.reset_index(drop = True)
    fwd['date'] = pd.to_datetime(fwd['date'], format='%Y-%m-%d')   
    fwd = fwd.groupby(by=["date"])['impl_strike'].mean().reset_index()
    fwd.rename(columns = {'impl_strike':'fwd_price'}, inplace = True)
    
    oppr_stock['strike_price'] = oppr_stock['strike_price'] / 1000 
    oppr_stock['date'] = pd.to_datetime(oppr_stock['date'], format='%Y-%m-%d')   
    oppr_stock['exdate'] = pd.to_datetime(oppr_stock['exdate'], format='%Y-%m-%d')   
    oppr_stock['days'] = (oppr_stock['exdate'] - oppr_stock['date']) / np.timedelta64(1, 'D')
    oppr_stock['price'] = (oppr_stock['best_bid'] + oppr_stock['best_offer']) / 2
    oppr_stock = oppr_stock[(oppr_stock['days'] > 50) & (oppr_stock['days'] < 150)] # CHANGE TO DESIRED MATURITY
    oppr_stock = oppr_stock[((oppr_stock['delta'] > -0.5) & (oppr_stock['delta'] < -0.05)) | ((oppr_stock['delta'] > 0.05) & (oppr_stock['delta'] < 0.5))]
    oppr_stock = oppr_stock.drop(['best_bid', 'best_offer', 'exdate'], axis=1)
    
    oppr_stock = pd.merge(oppr_stock, fwd, left_on = 'date', right_on = 'date')
    oppr_stock = pd.merge(oppr_stock, zcb, left_on = 'date', right_on = 'Date').drop(['Date'], axis=1)
    
    skew_stock = pd.DataFrame(columns=['date', 'secid', 'upper_skew', 'lower_skew', 'days'])
    for date in oppr_stock['date'].unique():
        temp = oppr_stock.loc[oppr_stock['date'] == date]
        if len(temp) > 0:
            skew_stock.loc[len(skew_stock)] = calc_upper_and_lower_skew(temp)
    
    skew_df = pd.concat([skew_df, skew_stock])

Wall time: 1h 23min 58s


In [82]:
skew_df = skew_df.reset_index(drop = True)#.drop(['Date'], axis=1)

In [84]:
skew_df = pd.merge(skew_df, sp500_secid_df, left_on = 'secid', right_on = 'secid')
skew_df = skew_df[(skew_df['upper_skew'] < 1000) & (skew_df['lower_skew'] < 1000)]
skew_df['total_skew'] = skew_df['upper_skew'] + skew_df['lower_skew']
skew = skew_df.groupby(by=["ticker","date"])[['upper_skew','lower_skew','total_skew']].sum().reset_index().pivot(index='date',columns='ticker')

In [87]:
skew_df

,date,secid,upper_skew,lower_skew,days,ticker,total_skew
3,2015-05-01,206814.0,0.003173,-0.023026,140.0,SEDG,-0.019854
5,2015-05-05,206814.0,0.001156,-0.042814,136.0,SEDG,-0.041658
6,2015-05-06,206814.0,0.004138,-0.036409,135.0,SEDG,-0.032271
7,2015-05-07,206814.0,0.00329,-0.038382,134.0,SEDG,-0.035092
8,2015-05-08,206814.0,0.002406,-0.052551,133.0,SEDG,-0.050145
...,...,...,...,...,...,...,...
123174,2021-12-27,216460.0,0.102543,-0.181522,53.0,TMC,-0.078979
123175,2021-12-28,216460.0,0.144486,-0.093216,52.0,TMC,0.051271
123176,2021-12-29,216460.0,0.130526,-0.079837,51.0,TMC,0.050689
123177,2021-12-30,216460.0,0.294134,-0.386979,141.0,TMC,-0.092845


In [89]:
skew_df.to_csv('skew_df.csv')